In [1]:
# hyper_parameter tunning

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras

In [5]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
from sklearn.datasets import fetch_california_housing

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
housing = fetch_california_housing()

In [10]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [12]:
class_names = ["T-shit/top", "Trouser", "Pullover", "Dress", "Coat",
              "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [13]:
scaler = StandardScaler()

In [14]:
X_train = scaler.fit_transform(X_train)

In [15]:
X_valid = scaler.transform(X_valid)

In [16]:
X_test = scaler.transform(X_test)

In [17]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [19]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [41]:
%%time
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 49us/sample - loss: 1.0950 - val_loss: 6.8181
Epoch 2/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.6794 - val_loss: 6.6763
Epoch 3/100
11610/11610 [==============================] - 0s 27us/sample - loss: 0.5748 - val_loss: 1.7621
Epoch 4/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.5017 - val_loss: 0.7749
Epoch 5/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4634 - val_loss: 0.4330
Epoch 6/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4455 - val_loss: 0.4966
Epoch 7/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4327 - val_loss: 0.4115
Epoch 8/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4242 - val_loss: 0.3952
Epoch 9/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.417

In [42]:
mse_test = keras_reg.score(X_test, y_test)

5160/5160 [==============================] - 0s 12us/sample - loss: 0.3817


In [43]:
X_new = X_test[:3]

In [44]:
y_pred = keras_reg.predict(X_new)

In [45]:
y_pred

array([0.8282322, 1.6953583, 3.8334901], dtype=float32)

In [46]:
from scipy.stats import reciprocal

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [48]:
# param_distribs = {
#     "n_hidden": [0, 1, 2, 3],
#     "n_neurons": np.arange(1, 100),
#     "learning_rate": reciprocal(3e-4, 3e-2)
# } 이게 원문인데 속도를 빨리하기위해서 설정

In [49]:
param_distribs = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(40, 45),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [50]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)

In [51]:
# %%time
# rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
#                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [52]:
import sklearn
sklearn.__version__

'0.23.2'

In [53]:
%%time
rnd_search_cv.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=2)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


7740/7740 [==============================] - 1s 65us/sample - loss: 0.8466 - val_loss: 0.5212
Epoch 2/10
7740/7740 [==============================] - 0s 31us/sample - loss: 0.4803 - val_loss: 5.1555
Epoch 3/10
3870/3870 [==============================] - 0s 13us/sample - loss: 0.4299
[CV]  learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41, total=   1.1s
[CV] learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


7740/7740 [==============================] - 1s 67us/sample - loss: 0.7795 - val_loss: 0.4920
Epoch 2/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4388 - val_loss: 0.3877
Epoch 3/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4080 - val_loss: 0.8254
Epoch 4/10
3870/3870 [==============================] - 0s 13us/sample - loss: 0.4266
[CV]  learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41, total=   1.4s
[CV] learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 1s 65us/sample - loss: 0.8765 - val_loss: 2.1839
Epoch 2/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4716 - val_loss: 2.1967
Epoch 3/10
3870/3870 [==============================] - 0s 14us/sample - loss: 0.4023
[CV]  learning_rate=0.010650914574384917, n_hidden=2, n_neurons=41, total=   1.1s
[CV] learning_rate=0.0184831486

3870/3870 [==============================] - 0s 13us/sample - loss: 0.5187
[CV]  learning_rate=0.0010466929964671095, n_hidden=1, n_neurons=41, total=   2.5s
[CV] learning_rate=0.0010466929964671095, n_hidden=1, n_neurons=41 ...
Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 0s 56us/sample - loss: 2.3471 - val_loss: 7.2693
Epoch 2/10
7740/7740 [==============================] - 0s 27us/sample - loss: 1.0115 - val_loss: 3.7503
Epoch 3/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.8258 - val_loss: 1.5519
Epoch 4/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7622 - val_loss: 0.7889
Epoch 5/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7229 - val_loss: 0.7609
Epoch 6/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6909 - val_loss: 1.1157
Epoch 7/10
3870/3870 [==============================] - 0s 12us/sample - loss: 0.6943
[CV]  l

Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 0s 57us/sample - loss: 3.4417 - val_loss: 5.2421
Epoch 2/10
7740/7740 [==============================] - 0s 28us/sample - loss: 2.0450 - val_loss: 4.3326
Epoch 3/10
7740/7740 [==============================] - 0s 27us/sample - loss: 1.4678 - val_loss: 3.3556
Epoch 4/10
7740/7740 [==============================] - 0s 27us/sample - loss: 1.1724 - val_loss: 2.4579
Epoch 5/10
7740/7740 [==============================] - 0s 28us/sample - loss: 1.0084 - val_loss: 1.7886
Epoch 6/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.9110 - val_loss: 1.3454
Epoch 7/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.8502 - val_loss: 1.0718
Epoch 8/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.8101 - val_loss: 0.9072
Epoch 9/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7815 - val_loss: 0.8095
Epoch 1

7740/7740 [==============================] - 0s 49us/sample - loss: 0.5787 - val_loss: 0.5359
Epoch 10/10
3870/3870 [==============================] - 0s 20us/sample - loss: 0.5470
[CV]  learning_rate=0.0007495594138208465, n_hidden=3, n_neurons=42, total=   4.7s
[CV] learning_rate=0.0007495594138208465, n_hidden=3, n_neurons=42 ...
Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 1s 153us/sample - loss: 3.0833 - val_loss: 6.6613
Epoch 2/10
7740/7740 [==============================] - 0s 49us/sample - loss: 1.2815 - val_loss: 5.8377
Epoch 3/10
7740/7740 [==============================] - 0s 49us/sample - loss: 0.9288 - val_loss: 3.8893
Epoch 4/10
7740/7740 [==============================] - 0s 48us/sample - loss: 0.7686 - val_loss: 2.7157
Epoch 5/10
7740/7740 [==============================] - 0s 50us/sample - loss: 0.6997 - val_loss: 1.9477
Epoch 6/10
7740/7740 [==============================] - 0s 51us/sample - loss: 0.6632 - val

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.1min finished


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000025C9CFCEA58>, as the constructor either does not set or modifies parameter learning_rate

In [54]:
rnd_search_cv.best_params_

{'learning_rate': 0.01848314861814289, 'n_hidden': 2, 'n_neurons': 43}

In [55]:
rnd_search_cv.score(X_test,y_test)

AttributeError: 'RandomizedSearchCV' object has no attribute 'scorer_'

In [ ]:
rnd_search_cv.best_estimator_

In [ ]:
model=rnd_search_cv